# 28 May 2025 ML model registration

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, weights=[0.9, 0.1], flip_y=0, random_state=42)
np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [15]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [16]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [19]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [22]:
# Initialize MLflow
mlflow.set_experiment("deployment model1")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/07/14 23:14:56 INFO mlflow.tracking.fluent: Experiment with name 'deployment model1' does not exist. Creating a new experiment.
2025/07/14 23:15:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/14 23:15:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/975587799656989030/runs/f823ee6c71e24543abf08fd0070bfd52.
2025/07/14 23:15:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/975587799656989030.
2025/07/14 23:15:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/14 23:15:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5000/#/experiments/97

### Register the Model

In [23]:
model_name = 'XGB-Classifier'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Classifier' already exists. Creating a new version of this model...
2025/07/14 23:24:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Classifier, version 3
Created version '3' of model 'XGB-Classifier'.
2025/07/14 23:24:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/975587799656989030/runs/367e4cd3ee7f4d5ea37b3d59debf18b3.
2025/07/14 23:24:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/975587799656989030.


### Load the Model

In [25]:
model_name = 'XGB-Classifier'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production server

In [27]:
current_model_uri = f"models:/{model_name}@appserver"
production_model_name = "finalproduction"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Registered model 'finalproduction' already exists. Creating a new version of this model...
Copied version '3' of model 'XGB-Classifier' to version '2' of model 'finalproduction'.


<ModelVersion: aliases=[], creation_timestamp=1752545840570, current_stage='None', description='', last_updated_timestamp=1752545840570, name='finalproduction', run_id='367e4cd3ee7f4d5ea37b3d59debf18b3', run_link='', source='models:/XGB-Classifier/3', status='READY', status_message='', tags={}, user_id='', version='2'>

In [ ]:
'''
# ignore below code

model_version = 2
prod_model_uri = f"models:/{production_model_name}@productionserver2"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]
'''                         

'\n# ignore below code\n\nmodel_version = 2\nprod_model_uri = f"models:/{production_model_name}@productionserver2"\n\nloaded_model = mlflow.xgboost.load_model(prod_model_uri)\ny_pred = loaded_model.predict(X_test)\ny_pred[:4]\n'